# Waarom vectoren, matrices en tensors?
Oorspronkelijk werden vectoren in de meetkunde en de mechanica geïntroduceerd om grootheden voor te stellen die zowel een grootte als een richting hebben, zoals verplaatsingen, krachten, versnelling, enz.
In dezelfde context werden matrices gebruikt om lineaire transformaties tussen vector-ruimtes voor te stellen.
**In Machine Learning worden vectoren, matrices en tensors veel algemener gebruikt als _containers_ voor data en model parameters** omwille van:
- computationele efficiëntie
- speciale tensors en tensor-operaties (bv. rotatie-matrices, convolutionele filters, decompositie, enz.)
- compacte notatie

## Toepassingen[^NotExhaustive]
[^NotExhaustive]: Dit zijn voorbeelden, dus niet als een exhaustieve lijst te beschouwen.
### Feature matrix voor tabulaire data
In veel gevallen vertrekken we van data in een **tabulair formaat** (denk aan een Excel sheet) waarbij iedere cel voor een bepaalde meting staat bij een bepaald item. Verschillende metingen komen overeen met verschillende kolommen en verschillende items met verschillende rijen.

In [ ]:
import os

import kagglehub
import pandas as pd


In [ ]:
# Download data
path = kagglehub.dataset_download("yashdevladdha/uber-ride-analytics-dashboard")

# Load data into Pandas DataFrame
csv_file = os.path.join(path, "ncr_ride_bookings.csv")
df = pd.read_csv(csv_file)
print("✅ Data loaded successfully!")

✅ Data loaded successfully!


In [3]:
df.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI



Machine Learning is een zaak van wiskunde en daarom moet alle informatie waarin we patronen willen herkennen vertaald worden naar numerieke waarden. Bij tabulaire data hebben we doorgaans te maken met heel verschillende data-types. Voor sommige data types is de vertaling in numerieke waarden evident, zoals temperatuur of aandelenkoersen, maar in andere gevallen zijn er speciale voorbereidingen nodig. **De manier waarop we informatie vertalen naar numerieke waarden bepaalt heel sterk welke patronen we al dan _niet_ kunnen oppikken met het ML model**. Het is onder andere belangrijk om rekening te houden met volgende aspecten:
- **Meetschaal**:
    - **Nominaal**: De informatie laat toe om items in bepaalde categorieën te plaatsen (bv. "fruit" $\neq$ "groente" $\neq$ "vlees" $\neq$ "rond" $\neq$ "vierkant" $\neq$ $\ldots$)
    - **Ordinaal**: De informatie laat toe om items te ordenen (bv. "small" $<$ "medium" $<$ "large")
    - **Interval**: De informatie laat toe om de afstand tussen items te bepalen (bv. $45\degree - 10\degree = 5\degree - (-30\degree)$)
    - **Ratio**: De informatie laat toe om de verhoudingen tussen items te bepalen (bv. $\frac{120km/h}{60km/h} = \frac{30km/h}{15km/h}$)

:::{note} Nominale waarden
Om nominale waarden om te zetten in numerieke waarden wordt vaak zogenaame **_one-hot encoding_** toegepast. Daarbij wordt iedere waarde omgezet in een binaire vector met evenveel dimensies als er _levels_ zijn in de nominale schaal.
:::

$$
\begin{pmatrix}
fruit \cr
fruit  \cr
groente \cr
vlees \cr
\vdots  \cr
\end{pmatrix} \rarr \begin{bmatrix}
1 & 0 & 0 \cr
1 & 0 & 0 \cr
0 & 1 & 0 \cr
0 & 0 & 1 \cr
\vdots  \cr
\end{bmatrix}
$$

- **Niet-lineariteit**:
    Op een interval of ratio schaal kunnen we aan de hand van niet lineaire tranformaties hetzelfde interval een ander gewicht geven naargelang de positie op de schaal. Voorbeelden zijn logarithmische transformatie, exponentiële transformatie en polynomiale transformatie.
:::{note} 🌍 Likes
:icon: false
:class: simple
Als je bijvoorbeeld een model wil maken dat verkoopscijfers voorspelt op basis van likes bij social-media marketing, zou je ervoor kunnen kiezen om een lineair regressie model te bouwen om de lineaire samenhang uit te drukken (te kwantificeren). In dit geval ga je echter de assumptie inbouwen dat het verschil tussen 1 of 2 likes hetzelfde is als tussen 100 000 en 200 000. In dat geval kan het zijn dat de linaire samenhang veel beter te vatten is door het aantal likes logarithmisch uit te drukken. 
:::

In [ ]:
## logarithmic demo here


- **Cyclische informatie**:
    Bij bepaalde schalen is er cycliciteit. Denk bijvoorbeeld aan de seizoenen ("winter" < "lente" < "zomer" < "herfst" < "winter" < ...) of dagen van de week ("zondag" < "maandag", "dinsdag", "woensdag", "donderdag", "vrijdag", "zaterdag", "zondag", "maandag", ...). Als we deze informatie zomaar vertalen naar numerieke waarden (bv. $[0, 1, 2, 3, 4, 5, 6, 0, 1, ...]^T$) maaken we een fout door bijvoorbeeld te stellen dat het verschil tussen "zaterdag" en "zondag" 6 is en tussen "zondag" en "maandag" 1 of dat "zaterdag" $>$ "zondag" en "zondag" $<$ "maandag".

- **_Missing_ data**:
    In veel tabulaire data sets kan niet voor ieder item steeds de volledige reeks observaties gemaakt worden (denk bijvoobeeld aan een temperatuursensor die een fout geeft). In sommige gevallen kunnen machine learning algorithmes daar perfect mee om (bv. Decision trees, Kalman Filters), maar veel algorithmes verwachten volledige input. Soms kan beslist worden om alle rijen te verwijderen waarvoor er een of meedere _missing values_ zijn - als daardoor nog genoeg data overblijft voor het leeralgorithme. Er van ook beslist worden om die _missing values_ op een zinvolle manier in te vullen (bv. door het gemiddelde van alle observaties). In dat geval spreekt men over **_missing value imputation_**.

Eens we de informatie waarin we patroonhekenning willen doen vertaald hebben naar numerieke waarden, verzamelen we ze in een _feature matrix/tensor_ met items in de rijen en verschillende features in de kolommen.

### Rotatie matrix